In [1]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from datetime import datetime 

import time

In [2]:
circle="Edc Bahraich"
month="jul2023"
month_doc=5
month_unpaid=7

In [3]:
def loop(file_name):
    return pd.read_csv(f"{month}/urban/{file_name}")

In [4]:
# df = loop('rapdrp_never_paid.csv')

In [5]:
def NonGOVT(df):
    govt = [30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,48,72,73,74,75,80,81,82,101,102,103,104,105,106,107,'80B','42A','41A','40A','H31'
           'H32','H33','H41','H42','H43','HV41','H13']
    govt = [str(x) for x in govt]
    df['gov_flag'] = df['SUPPLY_TYPE'].isin(govt)
    return df

In [6]:
def relGov(df):
#    ((df['gov_flag'] == False) & ( (pd.isnull(df['GOVT_CODE'] )))) 
    df['relGov'] = 'Goverment'
    df.loc[((df['gov_flag'] == False) & ( (pd.isnull(df['GOVT_CODE'] ))))  ,"relGov"] = 'Non-Goverment'
    return df

In [7]:
# def div(df):
#     govt = ["DIV354921","DIV354922","DIV355211","DIV355131"]
#     govt = [str(x) for x in govt]
#     df['div_flag'] = df['DIV_CODE'].isin(govt)
#     return df

In [8]:
# df=div(df)

In [9]:
# df['div_flag'].unique()

In [10]:
def arrear(df):
    
#     df = df[df['TOTAL_OUTSTANDING'] > 0]
    df['ARREAR_FILTER'] = None
    df.loc[df['TOTAL_OUTSTANDING'] < 5000,"ARREAR_FILTER"] = "<5k"
    df.loc[((df['TOTAL_OUTSTANDING'] >= 5000) & (df['TOTAL_OUTSTANDING'] < 10000)),"ARREAR_FILTER"] = ">=5k-10k"
    df.loc[((df['TOTAL_OUTSTANDING'] >= 10000) & (df['TOTAL_OUTSTANDING'] < 100000)),"ARREAR_FILTER"] = "10k-1L"
    df.loc[((df['TOTAL_OUTSTANDING'] >= 100000) & (df['TOTAL_OUTSTANDING'] < 5000000)),"ARREAR_FILTER"] = "1L-50L"
    df.loc[df['TOTAL_OUTSTANDING'] >= 5000000,"ARREAR_FILTER"] = "GRET_50L"
    
    return df

In [11]:
def billable(df):
    bil = ['In Service','TD','TD >6 MONTHS']
    df['bilable_flag'] = df['CON_STATUS'].isin(bil)
    return df
  

In [12]:
def td(df):
    bil = ['TD','TD >6 MONTHS']
    
    return df[df['CON_STATUS'].isin(bil)]

In [13]:
def CatLoad(df):
    df['LOAD'] = pd.to_numeric(df['LOAD'],errors="coerce")
    df['LOAD_FILTER'] = None
    df.loc[df['LOAD'] < 5,"LOAD_FILTER"] = "0-5kw"
    df.loc[((df['LOAD'] >= 5) & (df['LOAD'] < 10)),"LOAD_FILTER"] = "5-9kw"
    df.loc[df['LOAD'] >= 10,"LOAD_FILTER"] = "great than 10kw"
    return df

    

In [14]:
def faulty_Ageing(df):
    df['METER_READ_FLTY_CNT'] = pd.to_numeric(df['METER_READ_FLTY_CNT'],errors="coerce")
    df['faulty_Ageing'] = None
    df.loc[df['METER_READ_FLTY_CNT'] < 2,"faulty_Ageing"] = "faulty<2"
    df.loc[((df['METER_READ_FLTY_CNT'] >= 2) & (df['METER_READ_FLTY_CNT'] < 3)),"faulty_Ageing"] = "faulty>=2 TO <3"
    df.loc[df['METER_READ_FLTY_CNT'] >= 3,"faulty_Ageing"] = "faulty >=3"
    return df


In [15]:
def categories(df):
    #Supplytype.srs.startswith('1') for lmv1
    df['Cat'] = None
#     df[df['SUPPLY_TYPE'].str.startswith('1')] = "LMV1"
    df.loc[df['SUPPLY_TYPE'].str.startswith('1', na=False),"Cat"] = "LMV1"
    df.loc[df['SUPPLY_TYPE'].str.startswith('2',na=False),"Cat"] = "LMV2"
    df.loc[df['SUPPLY_TYPE'].str.startswith('3', na=False),"Cat"] = "LMV3"
    df.loc[df['SUPPLY_TYPE'].str.startswith('4', na=False),"Cat"] = "LMV4"
    df.loc[df['SUPPLY_TYPE'].str.startswith('5', na=False),"Cat"] = "LMV5"
    df.loc[df['SUPPLY_TYPE'].str.startswith('6', na=False),"Cat"] = "LMV6"
    df.loc[df['SUPPLY_TYPE'].str.startswith('7', na=False),"Cat"] = "LMV7"
    df.loc[df['SUPPLY_TYPE'].str.startswith('8', na=False),"Cat"] = "LMV8"
    df.loc[df['SUPPLY_TYPE'].str.startswith('9', na=False),"Cat"] = "LMV9"
#     df.loc[df['SUPPLY_TYPE'].str.startswith('10'),"Cat"] = "LMV10"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H1', na=False),"Cat"] = "HV1"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H2', na=False),"Cat"] = "HV2"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H3', na=False),"Cat"] = "HV3"
    df.loc[df['SUPPLY_TYPE'].str.startswith('H4', na=False),"Cat"] = "HV4"
    return df

In [16]:
def date(df):
    df['LAST_PAY_DATE'] = pd.to_datetime(df['LAST_PAY_DATE'],format='%d-%b-%Y',errors = "coerce")
    df['DOC'] = pd.to_datetime(df['DOC'],format='%d-%b-%Y',errors = "coerce")
    df['LAST_BILL_DATE'] = pd.to_datetime(df['LAST_BILL_DATE'],format='%d-%b-%Y',errors = "coerce")
    df['LAST_OK_READ_STATUS'] = pd.to_datetime(df['LAST_OK_READ_STATUS'],format='%d-%b-%Y',errors = "coerce")
    
    return df

In [17]:
# def unpaid(df):
#     df['unpaid_status'] = None
#     df.loc[df['LAST_PAY_DATE'] < '01-04-2023',"unpaid_status"] = "unpaid"
#     df.loc[(pd.isnull(df['LAST_PAY_DATE'])),"unpaid_status"] = "unpaid"     #df[~pd.isnull(df['Courses'])]
#     df.loc[df['LAST_PAY_DATE'] >= '01-04-2023',"unpaid_status"] = "paid"
    
#     return df
def unpaid(df):
    df['unpaid_status'] = None
    df.loc[df['LAST_PAY_DATE'] < pd.Timestamp(2023, 1, month_unpaid),"unpaid_status"] = "unpaid"
    df.loc[(pd.isnull(df['LAST_PAY_DATE'] )),"unpaid_status"] = "unpaid"     
    df.loc[df['LAST_PAY_DATE'] >= pd.Timestamp(2023, 1, month_unpaid),"unpaid_status"] = "paid"
    
    return df

In [18]:
# def Neverpaid(df):
#     df['Neverpaid_status'] = None

#     df.loc[((pd.isnull(df['LAST_PAY_DATE'])) & (df['DOC'] <'01-02-2023')),"Neverpaid_status"] = 1    #df[~pd.isnull(df['Courses'])]
#     df.loc[pd.isnull(df['Neverpaid_status']),"Neverpaid_status"] = 0
    
#     return df
def Neverpaid(df):
    df['Neverpaid_status'] = None
    df.loc[((pd.isnull(df['LAST_PAY_DATE'] )) & (df['DOC'] < pd.Timestamp(2023, 1, month_doc))),"Neverpaid_status"] = 1
    df.loc[pd.isnull(df['Neverpaid_status']),"Neverpaid_status"] = 0
    
    return df

In [19]:
import os

mylist = os.listdir(f"{month}/urban")

In [20]:
mylist


['MASTER_MVVNL_DIV354921_JUL_2023.csv',
 'MASTER_MVVNL_DIV354922_JUL_2023.csv',
 'MASTER_MVVNL_DIV355131_JUL_2023.csv',
 'MASTER_MVVNL_DIV355211_JUL_2023.csv']

In [21]:
start = time.time()
df = []
df = Parallel(n_jobs=-1, verbose=10,prefer="threads")(delayed(loop)(file_name) for file_name in mylist)
df = pd.concat(df, ignore_index=True)

end = time.time()
print("CSV:", end - start)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
C:\Users\daniy\AppData\Local\Temp\ipykernel_10052\1082892094.py:2: DtypeWarning: Columns (82) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/urban/{file_name}")
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.1s
C:\Users\daniy\AppData\Local\Temp\ipykernel_10052\1082892094.py:2: DtypeWarning: Columns (3,10,82) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/urban/{file_name}")
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    1.7s remaining:    1.7s
C:\Users\daniy\AppData\Local\Temp\ipykernel_10052\1082892094.py:2: DtypeWarning: Columns (3,10,82) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"{month}/urban/{file_name}")
C:\Users\daniy\AppData\Local\Temp\ipykernel_10052\1082892094.py:2: DtypeWarning: Columns (3,10,51,52,82) hav

CSV: 4.111170291900635


In [2]:
df.head(10)

NameError: name 'df' is not defined

In [23]:
df.DIV_CODE.unique()

array(['DIV354921', 'DIV354922', 'DIV355131', 'DIV355211'], dtype=object)

In [24]:
df['CON_STATUS'].unique()

array(['In Service', 'TD', 'Null', 'PD', 'NC Reject', 'TD Migrated',
       'COT', 'TD >6 MONTHS', 'PD Initiated', 'NC Pending',
       'COT Initiated', 'REJECTED', 'SB'], dtype=object)

In [25]:
column =list(df.columns)

In [26]:
column

['DIV_CODE',
 'SDO_CODE',
 'ACCT_ID',
 'KNO',
 'MOBILE_NO',
 'LANDLINE_NO',
 'BOOK_NO',
 'SCNO',
 'NAME',
 'ADDRESS',
 'SUPPLY_TYPE',
 'LOAD',
 'LOAD_UNIT',
 'DOC',
 'SECURITY_AMT',
 'CON_STATUS',
 'SERIAL_NBR',
 'MULTIPLY_FACTOR',
 'METER_STATUS',
 'LAST_BILL_DATE',
 'CLOSE_READING',
 'MDI',
 'BILL_BASIS',
 'BILL_TYP',
 'CONSUMPTION_CURR_MNTH',
 'CONSUMPTION_PREV_MNTH',
 'CONSUMPTION_PREV_TO_PREV_MNTH',
 'ARREAR',
 'LPSC',
 'CURRENT_ASSESSMENT',
 'CURRENT_CYCLE_LPSC',
 'TOTAL_OUTSTANDING',
 'DUE_DATE_REBATE',
 'LAST_OK_READING',
 'LAST_OK_READ_STATUS',
 'METER_READ_FLTY_CNT',
 'LAST_PAY_AMT',
 'LAST_PAY_DATE',
 'SUBSTATION',
 'FEEDER',
 'DT',
 'POLE_NO',
 'OPR_FLG',
 'BILL_AFTER_DATE',
 'METER_READ_REMARK',
 'INSTALLATION_DATE',
 'SBM_BILL_DATE',
 'SBM_MACHINE_ID',
 'BILL_CYC_CD',
 'TOWN',
 'SERVICE_CYC_CD',
 'CT_RATIO',
 'PT_RATIO',
 'EC',
 'FC',
 'REBATES',
 'MIN_CHARGES',
 'FUEL_SURCHARGE',
 'DEMAND_AMT',
 'LTMETERING_CHARGES',
 'CAP_CHARGES',
 'REG_SURCHARGE',
 'REG_SURCHARGE_2',


In [27]:
im_col = ['DIV_CODE',
          'SDO_CODE',
          'ACCT_ID',
          'SUBSTATION',
          'FEEDER',
    
          'NAME',
          'ADDRESS',
          'MOBILE_NO',
          'SUPPLY_TYPE',
          'CONSUMPTION_CURR_MNTH',
          'CONSUMPTION_PREV_MNTH',
          'DOC',
          'BILL_BASIS',
          'METER_STATUS',
          'METER_READ_REMARK',
          'CON_STATUS',
          'LOAD',
          'ARREAR',
          'CURRENT_ASSESSMENT',
          'TOTAL_OUTSTANDING',
          'LAST_OK_READING',
          'LAST_OK_READ_STATUS',
          'METER_READ_FLTY_CNT',
          'LAST_PAY_AMT',
          'LAST_PAY_DATE',
          'INDUSTRIAL_FLAG',
          'ACCT_INFLALTED_FLAG',
          'GOVT_CODE',
          'LAST_BILL_DATE',
          'PAYMENT_SRCE',
          'TOTAL_PAY_AMT']

In [28]:
df = df[im_col]

In [29]:
df = date(df)
df = unpaid(df)
df=Neverpaid(df)

In [30]:
df = NonGOVT(df)

In [31]:
df = relGov(df)

In [32]:
df = faulty_Ageing(df)

In [33]:
df = billable(df)


In [34]:
df['TOTAL_OUTSTANDING'] = pd.to_numeric(df['TOTAL_OUTSTANDING'],errors="coerce")
df = arrear(df)

In [35]:
df['ARREAR_FILTER'][:6]

0       <5k
1    10k-1L
2       <5k
3       <5k
4       <5k
5       <5k
Name: ARREAR_FILTER, dtype: object

In [36]:
df = categories(df)

In [37]:
df.head()

,DIV_CODE,SDO_CODE,ACCT_ID,SUBSTATION,FEEDER,NAME,ADDRESS,MOBILE_NO,SUPPLY_TYPE,CONSUMPTION_CURR_MNTH,...,PAYMENT_SRCE,TOTAL_PAY_AMT,unpaid_status,Neverpaid_status,gov_flag,relGov,faulty_Ageing,bilable_flag,ARREAR_FILTER,Cat
0,DIV354921,SDO3549212,9930810375,GULLABEER_34119,RAVIDAS_35492124119RAR,Mr MOHD AHMAD S/O Mr ALI MOHD,SALARGANJ NEAR - RAM LAL CHAKKI BAHRAICH Bahra...,9.795163e+09,10,463.0,...,COUNTER,6148.0,paid,0,False,Non-Goverment,None,True,<5k,LMV1
1,DIV354921,SDO3549212,7770804000,DEISEL POWER HOUSE_30165,DARGAH_35492120165DAR,MOHD ADIL KHAN,BUXIPURA BEHIND RAILWAY STATIONBAHRAICH BAHRAI...,6.307990e+09,10,183.0,...,COUNTER,2000.0,paid,0,False,Non-Goverment,None,True,10k-1L,LMV1
2,DIV354921,SDO3549212,9748257591,BASHIRGANJ_30166,KILLA FEEDER_35492120166KIL,CHUNNI W/O SHAUKAT ALI,QILA NEAR - MAHAJANI SCHOOL BAHRAICH Bahraich ...,9.415061e+09,10,99.0,...,NaN,NaN,paid,0,False,Non-Goverment,None,True,<5k,LMV1
3,DIV354921,SDO3549212,7773812957,GULLABEER_34119,TOWN-7_35492124119TO7,RAJU S/O LATE WASIM,MANSOORGANJ NEAR SHISHA MASJID BAHRAICH Bahra...,8.874348e+09,10,337.0,...,COUNTER,4700.0,paid,0,False,Non-Goverment,None,True,<5k,LMV1
4,DIV354921,SDO3549212,7715704000,BAXIPURA_34118,NAVA GRAHI_35492124118NAI,RANJEET KAUR,BALWANT SINGH GHASIARYIPURBARICH BAHRAICH UP IND,9.005692e+09,10,59.0,...,EWALET01,2000.0,paid,0,False,Non-Goverment,None,True,<5k,LMV1


In [38]:
# df['LAST_PAID_DATE'] = pd.to_datetime(df.LAST_PAID_DATE)
# df['SUPP_RELEASE_DATE'] = pd.to_datetime(df.SUPP_RELEASE_DATE)
# df['LAST_PAY_DATE'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y')) 
# df['DOC'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y')) 


In [39]:
df.dtypes


DIV_CODE                         object
SDO_CODE                         object
ACCT_ID                           int64
SUBSTATION                       object
FEEDER                           object
NAME                             object
ADDRESS                          object
MOBILE_NO                       float64
SUPPLY_TYPE                      object
CONSUMPTION_CURR_MNTH           float64
CONSUMPTION_PREV_MNTH           float64
DOC                      datetime64[ns]
BILL_BASIS                       object
METER_STATUS                     object
METER_READ_REMARK                object
CON_STATUS                       object
LOAD                            float64
ARREAR                          float64
CURRENT_ASSESSMENT              float64
TOTAL_OUTSTANDING               float64
LAST_OK_READING                 float64
LAST_OK_READ_STATUS      datetime64[ns]
METER_READ_FLTY_CNT             float64
LAST_PAY_AMT                    float64
LAST_PAY_DATE            datetime64[ns]


In [40]:
df.LAST_PAY_DATE.values

array(['2023-07-22T00:00:00.000000000', '2023-07-13T00:00:00.000000000',
       '2023-05-18T00:00:00.000000000', ...,
       '2023-07-26T00:00:00.000000000', '2023-04-20T00:00:00.000000000',
       '2023-06-13T00:00:00.000000000'], dtype='datetime64[ns]')

In [41]:
df=CatLoad(df)

In [42]:
# df1 =  df1[df1['bilable_flag'] == True]

In [43]:
# df1 = df[((df['gov_flag'] == False) & ( (pd.isnull(df['GOVT_CODE'] )))) ]

In [44]:
# df1=date(df1)

In [45]:
# df1 = df1[df1['DOC'] >= pd.Timestamp(2018, 4, 1)]

In [46]:
# df1 =  df[df['div_flag'] == True]

In [47]:
# df1 = df1[((df1['LAST_PAY_DATE'] < pd.Timestamp(2022, 12, 1)) | (pd.isnull(df['LAST_PAY_DATE'] )))]

In [48]:
# df1 = df1[((df1['gov_flag'] == False) & ( (pd.isnull(df['GOVT_CODE'] ))))]

In [49]:
# df1.head(10)

In [105]:
# df1 =  td(df1)

In [106]:
# df1.head(10)

,DIV_CODE,SDO_CODE,ACCT_ID,SUBSTATION,FEEDER,NAME,ADDRESS,MOBILE_NO,SUPPLY_TYPE,CONSUMPTION_CURR_MNTH,...,TOTAL_PAY_AMT,unpaid_status,Neverpaid_status,gov_flag,relGov,faulty_Ageing,bilable_flag,ARREAR_FILTER,Cat,LOAD_FILTER
20,DIV354921,SDO3549212,9781746000,BAXIPURA_34118,TOWN-5_35492124118TO5,RAHAT ALI,AMINPUR NAGROR BAHRAICH BAHRAICH UP IND,6.349853e+09,10,0.0,...,NaN,paid,0,False,Non-Goverment,None,True,<5k,LMV1,0-5kw
37,DIV354921,SDO3549212,3996332243,BAXIPURA_34118,TOWN-4_35492124118TO4,PUSHPA SAINE W/O CHOTEY LAL,CHAWNI SARKAR RAIPUR RAJA BAHRAICH Bahraich U...,9.628937e+09,10,159.0,...,NaN,paid,0,False,Non-Goverment,None,True,10k-1L,LMV1,0-5kw
38,DIV354921,SDO3549212,112078420,CIVIL LINE_34116,TELEPHONE_35492124116TPX,SAMIRUN W/O SIRAJUDDIN,SEKHDAHIR BAHRAICH Bahraich UP-271801 IND,8.081113e+09,10,52.0,...,NaN,unpaid,0,False,Non-Goverment,None,True,10k-1L,LMV1,0-5kw
49,DIV354921,SDO3549212,8670404000,HOSPITAL CHAURAHA_34114,STADIUAM_35492124114SAM,HARI BAHADUR,CIVIL LINES K D COMPLEX BAHRAICH UP IND,8.858650e+09,20,0.0,...,NaN,unpaid,0,False,Non-Goverment,None,True,10k-1L,LMV2,0-5kw
63,DIV354921,SDO3549212,3873360008,DEISEL POWER HOUSE_30165,GALLA MANDI_35492120165GAI,SMT SHAMIM BANO W/O MOHAMMAD IDRISH,DARGAH SHARIF NEAR POORVI GATE BAHRAICH Bahra...,9.792110e+09,10,80.0,...,NaN,unpaid,0,False,Non-Goverment,None,True,10k-1L,LMV1,0-5kw
71,DIV354921,SDO3549212,7191604000,CIVIL LINE_34116,TELEPHONE_35492124116TPX,RAM KUMAR SINGH,SARASWATI NAGAR OJHAARA MACHINE BAHRAICH UP IND,9.721158e+09,10,595.0,...,21000.0,paid,0,False,Non-Goverment,None,True,10k-1L,LMV1,0-5kw
88,DIV354921,SDO3549212,1119779326,DEISEL POWER HOUSE_30165,DARGAH_35492120165DAR,SMT SALIYA BEGUM W/O LATE LADDAN,MANSOORGANJ NEAR FAKEER MASJID BAHRAICH Bahra...,8.574051e+09,10,336.0,...,2144.0,paid,0,False,Non-Goverment,None,True,<5k,LMV1,0-5kw
113,DIV354921,SDO3549212,700846910,GULLABEER_34119,SALARGANJ_35492124119SAJ,SRI JAVED AHMAD S/O SRI CHUNNEY,SALARGANJ NEW BASTI Bahraich UP-271801 IND,9.839921e+09,10,102.0,...,4000.0,paid,0,False,Non-Goverment,None,True,>=5k-10k,LMV1,0-5kw
134,DIV354921,SDO3549212,2215604000,GULLABEER_34119,SALARGANJ_35492124119SAJ,RAM DHIRAJ CHAUHAN,PREM CHANDRA SALAR GANJBAHRAICH BAHRAICH UP IND,1.000000e+10,10,0.0,...,NaN,unpaid,0,False,Non-Goverment,None,True,10k-1L,LMV1,0-5kw
197,DIV354921,SDO3549212,4303804000,BAXIPURA_34118,TOWN-4_35492124118TO4,RAKESH KUMAR,RAIPUR RAJA JAILROAD BAHRAICH UP IND,9.794500e+09,10,630.0,...,4919.0,paid,0,False,Non-Goverment,None,True,<5k,LMV1,0-5kw


In [107]:
# df1.sort_values(by=['TOTAL_OUTSTANDING'], inplace=True,ascending=False)

In [108]:
# df1=df1.groupby('DIV_CODE').head(10).reset_index(drop=True)

In [50]:
start = time.time()
# df1.to_excel(f"{month}/top10-td-rapdrp.xlsx")
df.to_excel(f"{month}/gonda_rapdrp.xlsx") 

end = time.time()
print("CSV:", end - start)

CSV: 238.57917141914368


In [115]:

# # import pandas
# import pandas as pd
   
# # read csv data
# df1 = pd.read_csv('Student_data.csv')
# df2 = pd.read_csv('Course_enrolled.csv')
   
# Left_join = pd.merge(df1, 
#                      df2, 
#                      on ='Name', 
#                      how ='left')
# Left_join

In [116]:
print("jhgcgs")

jhgcgs


In [117]:
print(1+2)

3


In [118]:
def add (a,b):
    return a+b

In [119]:
add(4,6)

10